In [1]:
import os
import subprocess
import time
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# Disable SSL
import ssl
import googleapiclient.discovery

ssl._create_default_https_context = ssl._create_unverified_context

# Google API configuration
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = "client_secret.json"
TOKEN_FILE = "token.json"
YOUTUBE_MAX_DURATION = 43200  # 12 hours in seconds

MERGE_TIMEOUT = 2400  # 40 minutes
CONVERT_TIMEOUT = 7200  # 2 hours

def get_authenticated_service():
    """
    Authenticate with YouTube API and return a service object.
    If re-authentication is required, open a browser before starting the process.
    """
    creds = None

    # ✅ Check if a token file exists
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE)

    # ✅ If token is missing or invalid, refresh or re-authenticate before merging starts
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                print("🔄 Token expired. Attempting to refresh...")
                creds.refresh(Request())
                print("✅ Token refreshed successfully!")
            except Exception as e:
                print(f"⚠️ Token refresh failed: {e}\n🌐 Opening browser for re-authentication...")
                creds = None  # Force re-authentication

        if creds is None:
            print("🌐 Opening browser for authentication...")
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        # ✅ Save the new token immediately
        with open(TOKEN_FILE, "w") as token_file:
            token_file.write(creds.to_json())

    return build("youtube", "v3", credentials=creds)

# 🔥 Call authentication before any conversion or merging starts!
youtube_service = get_authenticated_service()
print("✅ Authentication successful! Now processing videos...")


def get_video_duration(video_path):
    """
    Returns the duration of a video in seconds.
    """
    try:
        result = subprocess.run(
            ["ffprobe", "-v", "error", "-show_entries", "format=duration",
             "-of", "default=noprint_wrappers=1:nokey=1", video_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        return float(result.stdout.strip())
    except Exception as e:
        print(f"⚠️ Warning: Could not read duration for {video_path}. Error: {e}")
        return None

MAX_VIDEO_DURATION = 21600  # 6 hours in seconds

def convert_videos(folder_path, video_files):
    """
    Converts videos to a uniform format (same codec, resolution, frame rate, etc.).
    Skips videos with no valid duration.
    """
    converted_folder = os.path.join(folder_path, "converted_videos")
    os.makedirs(converted_folder, exist_ok=True)
    converted_files = []

    # ✅ Filter out videos with no valid duration
    video_files = [f for f in video_files if get_video_duration(f) is not None]

    if not video_files:
        print(f"⚠️ No valid videos to convert in {folder_path}. Skipping...")
        return []

    print(f"🔄 Starting video conversion... (Timeout: {CONVERT_TIMEOUT // 3600} hours)")

    start_time = time.time()
    for video in video_files:
        converted_video = os.path.join(converted_folder, os.path.basename(video))

        try:
            subprocess.run(
                ["ffmpeg", "-i", video, 
                 "-vf", "scale=1280:720",  # ✅ Ensures all videos are 720p
                 "-r", "30",               # ✅ Sets frame rate to 30 FPS
                 "-c:v", "libx264", 
                 "-preset", "fast", 
                 "-crf", "23", 
                 "-c:a", "aac", 
                 "-b:a", "128k", 
                 "-strict", "experimental", 
                 converted_video],
                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=CONVERT_TIMEOUT
            )
        except subprocess.TimeoutExpired:
            print(f"⏳ Conversion **timed out** after {CONVERT_TIMEOUT // 3600} hours! Aborting this subfolder...")
            return []  # Return an empty list to indicate failure

        converted_files.append(converted_video)

        elapsed_time = time.time() - start_time
        if elapsed_time > CONVERT_TIMEOUT:
            print("⏳ Total conversion exceeded allowed time. Aborting conversion...")
            return []

    print("✅ Video conversion complete!")
    return converted_files

import urllib.parse

def merge_videos_gstreamer(folder_path, video_files):
    """
    Merges a list of video files using GStreamer, handling long paths.
    
    :param folder_path: Path to the folder containing videos.
    :param video_files: List of video file paths.
    :return: Path to merged video or None if merge fails.
    """
    if not video_files:
        print("❌ No video files provided for merging.")
        return None

    merged_video_path = os.path.join(folder_path, "merged_video.mp4")

    # Use absolute paths for the videos and properly encode spaces or special chars
    video_files = [f"file://{urllib.parse.quote(os.path.abspath(video))}" for video in video_files]

    # Construct the GStreamer pipeline
    gstreamer_command = [
        "gst-launch-1.0",
        *[item for video in video_files for item in ["filesrc", f"location={video}", "!", "decodebin"]],
        "concat", "!", "videoconvert", "!", "x264enc", "!", "mp4mux", "!", f"filesink location={merged_video_path}"
    ]

    print(f"🚀 Running merge command:\n{' '.join(gstreamer_command)}")

    try:
        result = subprocess.run(
            gstreamer_command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            check=True
        )

        if result.returncode == 0:
            print(f"✅ Merge successful: {merged_video_path}")
            return merged_video_path
        else:
            print(f"❌ GStreamer Merge Failed:\n{result.stderr}")
            return None

    except subprocess.CalledProcessError as e:
        print(f"❌ Exception during merge: {e}")
        return None

def merge_videos_and_create_timestamps(folder_path, video_files, part_number=None):
    """
    Merges videos, creates timestamps, and returns the final video path, description, and total duration.
    Splits the videos into parts if total duration exceeds 6 hours.
    """
    print(f"📂 Checking folder: {folder_path}")

    if not os.path.exists(folder_path):
        print(f"❌ Error: The folder path '{folder_path}' does not exist.")
        return None, None, None

    # ✅ Skip corrupted videos
    video_files = [f for f in video_files if get_video_duration(f) is not None]

    if not video_files:
        print(f"⚠️ No valid videos found in {folder_path}. Skipping folder...")
        return None, None, None

    total_duration = sum(get_video_duration(f) or 0 for f in video_files)

    # ✅ Split videos if total duration > 6 hours
    if total_duration > MAX_VIDEO_DURATION:
        print(f"⚠️ Video duration too long ({total_duration} sec). Splitting into two parts...")
        mid_index = len(video_files) // 2
        first_half = video_files[:mid_index]
        second_half = video_files[mid_index:]

        merged_video_path_1 = try_merge_videos(folder_path, first_half, sum(get_video_duration(f) or 0 for f in first_half), merge_attempt=1)
        merged_video_path_2 = try_merge_videos(folder_path, second_half, sum(get_video_duration(f) or 0 for f in second_half), merge_attempt=2)

        if merged_video_path_1 and merged_video_path_2:
            return [merged_video_path_1, merged_video_path_2], "\n".join(os.path.basename(f) for f in video_files), total_duration
        else:
            return None, None, None

    # ✅ Try merging normally if duration is within limit
    merged_video_path = try_merge_videos(folder_path, video_files, total_duration, merge_attempt=1)
    if merged_video_path:
        return merged_video_path, "\n".join(os.path.basename(f) for f in video_files), total_duration

    print("🔄 Converting videos due to merge failure...")
    converted_files = convert_videos(folder_path, video_files)

    if not converted_files:
        print(f"❌ Conversion failed for {folder_path}. Skipping upload.")
        return None, None, None

    return try_merge_videos(folder_path, converted_files, total_duration, merge_attempt=2), "\n".join(os.path.basename(f) for f in converted_files), total_duration

def upload_to_youtube(video_path, title, description, folder_path):
    """
    Uploads a video to YouTube and saves the video URL in 'youtube link.txt'.
    """
    print(f"📤 Uploading: {title}...")
    youtube = get_authenticated_service()
    request = youtube.videos().insert(
        part="snippet,status",
        body={"snippet": {"title": title, "description": description, "categoryId": "27"}, "status": {"privacyStatus": "unlisted"}},
        media_body=MediaFileUpload(video_path, chunksize=-1, resumable=True)
    )

    _, response = request.next_chunk()
    video_url = f"https://www.youtube.com/watch?v={response['id']}"
    with open(os.path.join(folder_path, "youtube_link.txt"), "w") as f:
        f.write(video_url + "\n")

    print(f"✅ Upload complete! Video URL: {video_url}")
    return video_url

if __name__ == "__main__":
    parent_folder = input("📂 Enter the parent folder containing all video subfolders: ").strip()

    for subfolder in sorted(os.listdir(parent_folder)):
        subfolder_path = os.path.join(parent_folder, subfolder)

        if os.path.isdir(subfolder_path):
            print(f"🚀 Processing folder: {subfolder_path}")

            video_files = sorted([os.path.join(subfolder_path, f) for f in os.listdir(subfolder_path) if f.endswith('.mp4')])
            merged_video_path, description, _ = merge_videos_and_create_timestamps(subfolder_path, video_files)
            if merged_video_path:
                upload_to_youtube(merged_video_path, subfolder, description, subfolder_path)

print("✅ All subfolders processed!")


✅ Authentication successful! Now processing videos...
🚀 Processing folder: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\Premiere Pro\InfiniteSkills\Learning Adobe Premiere Pro CS6 Part 01
📂 Checking folder: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\Premiere Pro\InfiniteSkills\Learning Adobe Premiere Pro CS6 Part 01
⚠️ Video duration too long (29710.107266 sec). Splitting into two parts...


NameError: name 'try_merge_videos' is not defined